# Extracción de Datos API RIOT

* Implementar técnicas de extracción de datos por medio de Python.

* Implementar técnicas de almacenamiento de datos con el formato Delta Lake

## Paso 1: Conseguir a los jugadores mejores rankeados

Este paso debe abarcarse mediante el endpoint del API LEAGUE-EXP-V4, el cual tiene 3 **PATH PARAMETERS** queue, tier y division y un **QUERY PARAMETERS** de interes que es la región.

### PATH PARAMETERS:

1. Parametro "queue", es donde se especifica los modos competitivos (ranked) que Riot maneja en la API, y están asociados a juegos distintos de su ecosistema las cuales son:

    * RANKED_SOLO_5x5: Solo/Dúo en Summoner’s Rift. El modo competitivo más popular: partidas 5 vs 5, donde juegas solo o con un amigo, con emparejamiento según MMR y rango.

    * RANKED_TFT: Flex en Summoner’s Rift. También 5 vs 5, pero puedes entrar en premade de hasta 5 jugadores. Se usa un MMR diferente al Solo/Dúo.

    * RANKED_FLEX_SR: Flex en Twisted Treeline (3 vs 3). Este mapa fue retirado en 2019, pero en la API todavía existe por compatibilidad con datos antiguos.

    * RANKED_FLEX_TT: Modo competitivo de TFT. Partidas automáticas por rondas, donde compites contra 7 jugadores para armar la mejor composición.

2. Parametro "tier" en la API de Riot indica el nivel de liga dentro del sistema de clasificatorias (Ranked) para el modo de juego que elijas (queue). El cual sigue la jerarquia: IRON < BRONZE < SILVER < GOLD < PLATINUM < EMERALD < DIAMOND < MASTER < GRANDMASTER < CHALLENGER.

    * Solo los tiers de Iron a Diamond se dividen en 4 divisiones: IV (peor) → III → II → I (mejor dentro del tier).

    * Master, Grandmaster y Challenger no tienen divisiones, solo LP (League Points).

3. El parametro division que aplica desde el nivel Iron hasta Diamante.


![players](Imagenes/api_riot_games_players.png)


In [1]:
import pandas as pd
import requests

In [ ]:
# Parámetros iniciales
api_key = "RGAPI-13690e27-1a56-491d-9c34-f101545b873d"   
region  = ["br1", "eun1", "euw1", "jp1", "kr", "la1", 
           "la2", "me1", "na1", "oc1", "ru", "sg2", 
           "tr1", "tw2", "vn2"]

tier_endpoints = {
    "CHALLENGER": "challengerleagues",
    "GRANDMASTER": "grandmasterleagues",
    "MASTER": "masterleagues"
}

# DataFrame vacío para acumular resultados
df_final = pd.DataFrame()

for r in region:
    for tier_name, endpoint in tier_endpoints.items():
        # Construir la URL del endpoint
        url = f"https://{r}.api.riotgames.com/lol/league/v4/{endpoint}/by-queue/RANKED_SOLO_5x5"

        # Header con la API key
        headers = {"X-Riot-Token": api_key}

        # Hacer la solicitud
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
        # Normalizar solo la lista de jugadores
            df_region = pd.json_normalize(data["entries"])
            
            # Agregar columna de región
            df_region["region"] = r
            df_region["tier"] = tier_name
            
            # Acumular al DataFrame final
            df_final = pd.concat([df_final, df_region], ignore_index=True)
            
            print(f"Región {r} - {tier_name} procesada. Total acumulado: {len(df_final)} registros")
        else:
            print("Error:", response.status_code)

print("Total final de jugadores:", len(df_final))

## Paso 2: Conseguir las estadísticas de los jugadores obtenidos en el paso 1 dado los campeones que utiliza

Los campos que se obtienen mediante esta consulta son:

![Campeones](Imagenes/api_riot_games_champions.png)


In [ ]:
import time

api_key = "RGAPI-13690e27-1a56-491d-9c34-f101545b873d"

def get_champion_mastery(puuid:str, region:str, api_key:str):

    url = f"https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"
    headers = {"X-Riot-Token": api_key}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code} para el jugador {puuid} en región {region}")
        return None

df_list = [] # Es menos costoso qu

for _, row in df_final.iterrows():
    puuid  = row["puuid"]
    region = row["region"]
    mastery_data = get_champion_mastery(puuid, region, api_key)

    if mastery_data:
        df_player = pd.json_normalize(mastery_data)
        df_player["puuid"] = puuid
        df_player["region"] = region
        df_list.append(df_player)
    
    time.sleep(1.2)

df_players = pd.concat(df_list, ignore_index=True)

In [15]:
api_key = "RGAPI-13690e27-1a56-491d-9c34-f101545b873d"
puuid = "D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvyuR3KqQIPi58IPoQ28RryV0qr14ihQrlQ"
region = "br1"

url = f"https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"
headers = {"X-Riot-Token": api_key}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.json())

pd.json_normalize(response.json())

200
[{'puuid': 'D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvyuR3KqQIPi58IPoQ28RryV0qr14ihQrlQ', 'championId': 76, 'championLevel': 9, 'championPoints': 69371, 'lastPlayTime': 1755396152000, 'championPointsSinceLastLevel': 4771, 'championPointsUntilNextLevel': 6229, 'markRequiredForNextLevel': 2, 'tokensEarned': 30, 'championSeasonMilestone': 2, 'milestoneGrades': ['A-'], 'nextSeasonMilestone': {'requireGradeCounts': {'S-': 1}, 'rewardMarks': 2, 'bonus': False, 'totalGamesRequires': 1}}, {'puuid': 'D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvyuR3KqQIPi58IPoQ28RryV0qr14ihQrlQ', 'championId': 126, 'championLevel': 7, 'championPoints': 50164, 'lastPlayTime': 1754786212000, 'championPointsSinceLastLevel': 7564, 'championPointsUntilNextLevel': 3436, 'markRequiredForNextLevel': 1, 'tokensEarned': 25, 'championSeasonMilestone': 4, 'milestoneGrades': ['A-', 'B+'], 'nextSeasonMilestone': {'requireGradeCounts': {'S-': 2}, 'rewardMarks': 1, 'bonus': True, 'totalGamesRequires': 2}}, {'puuid': 'D_

,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,markRequiredForNextLevel,tokensEarned,championSeasonMilestone,milestoneGrades,nextSeasonMilestone.requireGradeCounts.S-,nextSeasonMilestone.rewardMarks,nextSeasonMilestone.bonus,nextSeasonMilestone.totalGamesRequires,nextSeasonMilestone.requireGradeCounts.A-
0,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,76,9,69371,1755396152000,4771,6229,2,30,2,[A-],1.0,2,False,1,NaN
1,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,126,7,50164,1754786212000,7564,3436,1,25,4,"[A-, B+]",2.0,1,True,2,NaN
2,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,91,7,43966,1755826974000,1366,9634,1,21,5,"[A, S-]",2.0,1,True,2,NaN
3,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,64,6,36314,1755027615000,4714,6286,1,21,3,"[A, A+]",1.0,2,False,1,NaN
4,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,106,6,34164,1754172670000,2564,8436,1,18,3,"[B-, B+]",1.0,2,False,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,233,1,325,1752771174000,325,1475,0,0,0,[C],NaN,1,False,1,1.0
102,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,90,1,317,1739467485000,317,1483,0,0,0,NaN,NaN,1,False,1,1.0
103,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,804,1,277,1755291551000,277,1523,0,0,0,[B],NaN,1,False,1,1.0
104,D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvy...,10,1,276,1753737678000,276,1524,0,0,0,[C],NaN,1,False,1,1.0


## Paso 3: Obtener a los campeones (Personajes) existentes en el juego.


In [ ]:
url = "https://ddragon.leagueoflegends.com/cdn/15.15.1/data/en_US/champion.json"
response = requests.get(url)
champions_data = response.json()

df = pd.json_normalize(champions_data["data"].values())
df.drop(columns="version", inplace=True)

In [6]:


# Iterar sobre las filas con iterrows()
for _, row in df_final.iterrows():
    print(f"Puuid: {row['puuid']}, Edad: {row['region']}")




Puuid: mpRcRbIMxX-lqjvhUIcEWMFb4piC2aKyep2lguMXXsgjwpOi-bgO7f_pSojVnc5hzBbFSP0BTcUmrQ, Edad: br1
Puuid: tXloYdO7NZWWlrQHwfHT1rsCFzLMhFQd93jSJRtKnOYh5bjT31zVT4wxGNOC2Dt_ZztKnUFpa_jeVA, Edad: br1
Puuid: D_Y1yJT9ZDrBJBjKr8UOd0PZoMe3g3E7nAs3z8tByuarvyuR3KqQIPi58IPoQ28RryV0qr14ihQrlQ, Edad: br1
Puuid: L6EDGg5L7GkW5A8PXimnLLUHy49hNmppsaT5EqYM1-VdMhw2IkUaN3SUBBforjAXsGUGsKpSzDz8ow, Edad: br1
Puuid: A7QxN1AE2JID8LTdMQztzenhHjwOcXJfFPyqCjeVvtO3ZbrroRLXeqcZQ7eXtHtOY1WNBS8G4QKBng, Edad: br1
Puuid: NwPoYoS3Xtv-rU1nyzwKNXTQNjzSz9-vvRDcoMjZRAe-Kkm0SCCJw9jfVQ17nqVM_vmyp0Ap5Gb4GQ, Edad: br1
Puuid: K0INdSpMA9GpXZKKJQkqtw6OWyEa8r6Jxv82NEXReGIX2vXop76J41ey1WeYAEbDCeLgq6bf4s9OnA, Edad: br1
Puuid: Qtig1J-ic6A92yctnwEWs--tpfPaOY4g-47y0R-Af0ZZtEhXduvrYj0AoTxSn0ehFWplT8bWdObKuQ, Edad: br1
Puuid: IxbZTAcKY3Fi0VBUCJR5eMlO-Tfi825KnLRq4U9SXzMnBJQCJCh2j9KigH8RmNG42zQm9xyQHe32wg, Edad: br1
Puuid: cBaDh3XxonXmAOtWWx7MLxeII0FfbmsC7yT2OB5kjDuMxmRljWTicBCvI4TQyYL9CU8McAASQMXddQ, Edad: br1
Puuid: n-3AvSoUGITZiYMdeomAoZQ

KeyboardInterrupt: 